In [1]:
import transformers
import torch
import pandas as pd

/home/bracke/miniconda3/envs/gpu-venv-transnormer/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set-up device, tokenizer, model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = transformers.T5ForConditionalGeneration.from_pretrained("/home/bracke/code/transnormer/models/models_2023-06-12/checkpoint-15000").to(device)
# model = transformers.T5ForConditionalGeneration.from_pretrained("/home/bracke/code/transnormer/models/models_2023-06-13/checkpoint-800").to(device)
# model_orig = transformers.T5ForConditionalGeneration.from_pretrained("google/byt5-small").to(device)
tokenizer = transformers.AutoTokenizer.from_pretrained("google/byt5-small")

In [9]:
# Input string
s_orig = "Nach etlichen Taͤntzen kam vnſer Præſidentin wider daher/ vnd vermeldet vns wie das ſich die Kuͤnſtler vnd Studioſi"# gegen jhrer Koͤnig. Majeſt. erbotten/ deren zu ehren/ vnd gefallen vor dero abzug/ ein froͤliche Comœdiam  zu agieren/ wolten nuhn wir derſelben auch beywohnen/ vnd Koͤnig. Ma. auff der Sonnen Hauß begleiten/ daß were dero Lieb/ vnd wolte ſolches in allen gnaden erkennen:"
s_norm = "Nach etlichen Tänzen kam unser Präsidentin wider daher/ und vermeldet uns wie das sich die Künstler und Studiosi"# gegen ihrer König. Majest. erboten/ deren zu ehren/ und gefallen vor deren Abzug/ ein fröhliche Comoediam zu agieren/ wollten nun wir derselben auch beiwohnen/ und König. Ma. auf der Sonnen Haus begleiten/ dass wäre deren Liebe/ und wollte solches in allen Gnaden erkennen:" 
s_orig = "Mercks Wienn/ Das iſt: Deß wuͤtenden Todts Ein umſtaͤndige Beſchreibung/ Jn Der beruͤhmten Haupt- und Kaͤiſerl."
s_norm = "Mercks Wien/ Das iſt: Des wütenden Tods Ein umständige Beschreibung/ In Der berühmten Haupt- und Kaiserl."
s_norm = "Merks Wien/ Das ist: Des wütenden Tods Ein umständige Beschreibung/ In Der berühmten Haupt- und Käiserl."
s_orig = 'Weilen des menschen Lebent ist kürtz seine gedechtnüs Schwag, und die Sinne zum Misverstendtnüs stets geneiget (...)'
s_norm = "DUMMY"

inputs = tokenizer(s_orig, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
outputs = tokenizer(s_norm, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
input_ids = inputs.input_ids.to(device)
output_ids = outputs.input_ids.to(device)
output_ids[output_ids==0] = -100 # replace padding token with -100
attention_mask = inputs.attention_mask.to(device)
example = {
    "input_ids":input_ids,
    "attention_mask":attention_mask,
    "labels":output_ids
}

# Loss
output_forward = model(**example)
print(output_forward.loss)
# output_forward = model_orig(**example)
# print(output_forward.loss)

tensor(18.5427, device='cuda:0', grad_fn=<NllLossBackward0>)


In [10]:

# Generation
output_gen = model.generate(
    input_ids, 
    attention_mask=attention_mask, 
    max_length=512,
    no_repeat_ngram_size=0,
    # early_stopping = True,
    length_penalty=2.0,
    num_beams=3,
    )
# del input_ids, attention_mask # free memory
outputs_as_str = tokenizer.batch_decode(output_gen, skip_special_tokens=True)
print(outputs_as_str[0])

Weilen des Menschen Lebend ist kürz seine Gedechtnis Schwag, und die Sinne zum Misverstendnis stets geneigt (...)"


Byte: Weilen des Menschen Lebend ist kürz seine Gedechtnis Schwag, und die Sinne zum Misverstendnis stets geneigt (...)
Subword: Weilen des Menschen Lebent ist kürz seine Gedächtnis Schwag, und die Sinne zum Missverstendtnis stets geneigt (... )
Gold: Weilen des Menschen Leben ist kurz seine Gedächtnis schwach, und die Sinne zum Missverständnis stets geneigt

---

## Sentences from the train set

`orig:` 'Nach etlichen Taͤntzen kam vnſer Præſidentin wider daher/ vnd vermeldet vns wie das ſich die Kuͤnſtler vnd Studioſi gegen jhrer Koͤnig. Majeſt. erbotten/ deren zu ehren/ vnd gefallen vor dero abzug/ ein froͤliche Comœdiam zu agieren/ wolten nuhn wir derſelben auch beywohnen/ vnd Koͤnig. Ma. auff der Sonnen Hauß begleiten/ daß were dero Lieb/ vnd wolte ſolches in allen gnaden erkennen:'

`CAB-norm:` 'Nach etlichen Tänzen kam unser Präsidentin wider daher/ und vermeldet uns wie das sich die Künstler und Studiosi gegen ihrer König. Majest. erboten/ deren zu ehren/ und gefallen vor deren Abzug/ ein fröhliche Comoediam zu agieren/ wollten nun wir derselben auch beiwohnen/ und König. Ma. auf der Sonnen Haus begleiten/ dass wäre deren Liebe/ und wollte solches in allen Gnaden erkennen:'

In [11]:
my_string = r"Mercks Wienn/ Das i\u017ft: De\u00df wu\u0364tenden Todts Ein um\u017fta\u0364ndige Be\u017fchreibung/ Jn Der beru\u0364hmten Haupt- und Ka\u0364i\u017ferl."
bytes(my_string, "utf-8").decode("unicode_escape")

'Mercks Wienn/ Das iſt: Deß wuͤtenden Todts Ein umſtaͤndige Beſchreibung/ Jn Der beruͤhmten Haupt- und Kaͤiſerl.'